In [1]:
! nvidia-smi # 需要使用GPU运行

Mon Sep 11 17:35:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          On   | 00000000:00:07.0 Off |                    0 |
|  0%   43C    P8    21W / 150W |      0MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install openai-whisper
! pip install modelscope
! pip install tts-autolabel -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install typeguard==2.3.1
! pip install sox
! pip install bitstring
! pip install pysptk --no-build-isolation
! pip install kantts -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install pytorch_wavelets
! pip install tensorboardX

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 2.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.9 MB/s eta 0:00:00a 0:00:01
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796921 sha256=43bb978ba391573b908647d86d27a0ff5e11ba8a446b199cb92d4e2e1f569e3b
  Stored in directory: /root/.cache/pip/wheels/d4/ff/33/8ebeb27b8b4065772cf1ba1ac0799b0b78c5e69bcf5aa0ad1c
Successfully built openai-whisper
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to 

In [4]:
! git clone https://ghproxy.com//https://github.com/fbcotter/pytorch_wavelets
! pip install matplotlib
! pip install numpy==1.22.0

正克隆到 'pytorch_wavelets'...
remote: Enumerating objects: 978, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 978 (delta 79), reused 91 (delta 46), pack-reused 836
接收对象中: 100% (978/978), 6.80 MiB | 844.00 KiB/s, 完成.
处理 delta 中: 100% (663/663), 完成.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 3.5 MB/s eta 0:00:0000:0100:01
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pi

In [5]:
!apt-get install sox

正在读取软件包列表... 完成%
正在分析软件包的依赖关系树       
正在读取状态信息... 完成         
sox 已经是最新版 (14.4.2+git20190427-2+deb11u2build0.20.04.1)。
升级了 0 个软件包，新安装了 0 个软件包， 要卸载 0 个软件包，有 0 个软件包未被升级。


In [6]:
cd pytorch_wavelets

/mnt/workspace/pytorch_wavelets


In [7]:
pip install .

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Processing /mnt/workspace/pytorch_wavelets
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-wavelets: filename=pytorch_wavelets-1.3.0-py3-none-any.whl size=54869 sha256=1fc49cfc0b01a19413e6be4436062ed469278847835cb05dccbb801f77d3faca
  Stored in directory: /tmp/pip-ephem-wheel-cache-8f_rgmle/wheels/3c/e7/94/b09b4190b59d9215140c4e378232705db50eee93499c924e81
Successfully built pytorch-wavelets
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: pytorch-wavelets
    Found existing installation: pytorch-wavelets 1.3.0
    Uninstalling pytorch-wavelets-1.3.0:
    

In [ ]:
import os
os._exit(00) # 重启notebook

In [1]:
import sox

对音频切片处理

In [2]:
import subprocess
from pathlib import Path
import librosa
from scipy.io import wavfile
import numpy as np
import torch
import csv
import whisper

def split_long_audio(model, filepaths, character_name, save_dir="data_dir", out_sr=44100):
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    for file_idx, filepath in enumerate(filepaths):

        save_path = Path(save_dir) / character_name
        save_path.mkdir(exist_ok=True, parents=True)

        print(f"Transcribing file {file_idx}: '{filepath}' to segments...")
        result = model.transcribe(filepath, word_timestamps=True, task="transcribe", beam_size=5, best_of=5)
        segments = result['segments']

        wav, sr = librosa.load(filepath, sr=None, offset=0, duration=None, mono=True)
        wav, _ = librosa.effects.trim(wav, top_db=20)
        peak = np.abs(wav).max()
        if peak > 1.0:
            wav = 0.98 * wav / peak
        wav2 = librosa.resample(wav, orig_sr=sr, target_sr=out_sr)
        wav2 /= max(wav2.max(), -wav2.min())

        for i, seg in enumerate(segments):
            start_time = seg['start']
            end_time = seg['end']
            wav_seg = wav2[int(start_time * out_sr):int(end_time * out_sr)]
            wav_seg_name = f"{character_name}_{file_idx}_{i}.wav"
            out_fpath = save_path / wav_seg_name
            wavfile.write(out_fpath, rate=out_sr, data=(wav_seg * np.iinfo(np.int16).max).astype(np.int16))

/opt/conda/lib/python3.8/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [4]:
whisper_size = "medium"
whisper_model = whisper.load_model(whisper_size)

100%|█████████████████████████████████████| 1.42G/1.42G [01:30<00:00, 17.0MiB/s]


请在下方{filename}处填写您上传的wav文件名

In [6]:
split_long_audio(whisper_model, "原始克隆前的声音.WAV", "test", "dataset_raw") # 请在{filename}处填写您上传的wav文件名

Transcribing file 0: '原始克隆前的声音.WAV' to segments...


## 需要新建三个文件夹，分别是："test_wavs", "output_training_data", "pretrain_work_dir"

In [7]:
! mv  -v ./dataset_raw/test/* ./test_wavs/

已重命名 './dataset_raw/test/test_0_0.wav' -> './test_wavs/test_0_0.wav'
已重命名 './dataset_raw/test/test_0_1.wav' -> './test_wavs/test_0_1.wav'
已重命名 './dataset_raw/test/test_0_2.wav' -> './test_wavs/test_0_2.wav'


In [8]:
from modelscope.tools import run_auto_label

2023-09-11 18:00:51,910 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-09-11 18:00:51,912 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2023-09-11 18:00:51,912 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-09-11 18:00:51,913 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-09-11 18:00:51,952 - modelscope - INFO - Loading done! Current index file version is 1.9.0, with md5 e024469f80e1732f2cfc5efd837ca7df and a total number of 921 components indexed


In [11]:
input_wav = "./test_wavs/"
output_data = "./output_training_data/"
ret, report = run_auto_label(input_wav=input_wav, work_dir=output_data, resource_revision="v1.0.7")

2023-09-11 18:02:43,458 - modelscope - INFO - Use user-specified model revision: v1.0.7
Downloading: 100%|██████████| 185/185 [00:00<00:00, 33.5kB/s]
Downloading: 100%|██████████| 706M/706M [00:02<00:00, 254MB/s]  
Downloading: 100%|██████████| 6.24k/6.24k [00:00<00:00, 1.87MB/s]
INFO:root:2023-09-11 18:02:53
INFO:root:TTS-AutoLabel version: 1.1.8
INFO:root:TTS-AutoLabel resource path: /mnt/workspace/.cache/modelscope/damo/speech_ptts_autolabel_16k/model
INFO:root:Target sampling rate: 16000
INFO:root:Input wav dir: /mnt/workspace/test_wavs
INFO:root:Output data dir: /mnt/workspace/output_training_data
INFO:root:wav_preprocess start...
INFO:root:---  new folder...  ---
INFO:root:---  OK  ---
100%|██████████| 3/3 [00:00<00:00, 86.74it/s]
INFO:root:[VAD] chunk recordings for training.
INFO:root:wav cut by vad start...
  0%|          | 0/3 [00:00<?, ?it/s]INFO:root:wav test_0_2_S0000 is no need to cut
INFO:root:wav test_0_1_S0000 is no need to cut
INFO:root:wav test_0_0_S0000 is no need t

训练 Training

In [12]:
from modelscope.metainfo import Trainers
from modelscope.trainers import build_trainer
from modelscope.utils.audio.audio_utils import TtsTrainType

pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_16k'

dataset_id = "./output_training_data/"
pretrain_work_dir = "./pretrain_work_dir/"

# 训练信息，用于指定需要训练哪个或哪些模型，这里展示AM和Vocoder模型皆进行训练
# 目前支持训练：TtsTrainType.TRAIN_TYPE_SAMBERT, TtsTrainType.TRAIN_TYPE_VOC
# 训练SAMBERT会以模型最新step作为基础进行finetune
train_info = {
    TtsTrainType.TRAIN_TYPE_SAMBERT: {  # 配置训练AM（sambert）模型
        'train_steps': 202,               # 训练多少个step
        'save_interval_steps': 200,       # 每训练多少个step保存一次checkpoint
        'log_interval': 10               # 每训练多少个step打印一次训练日志
    }
}

# 配置训练参数，指定数据集，临时工作目录和train_info
kwargs = dict(
    model=pretrained_model_id,                  # 指定要finetune的模型
    model_revision = "v1.0.6",
    work_dir=pretrain_work_dir,                 # 指定临时工作目录
    train_dataset=dataset_id,                   # 指定数据集id
    train_type=train_info                       # 指定要训练类型及参数
)

trainer = build_trainer(Trainers.speech_kantts_trainer,
                        default_args=kwargs)

trainer.train()

2023-09-11 18:03:12,154 - modelscope - INFO - Use user-specified model revision: v1.0.6
2023-09-11 18:03:13,973 - modelscope - INFO - Use user-specified model revision: v1.0.6
2023-09-11 18:03:14,540 - modelscope - INFO - Set workdir to ./pretrain_work_dir/
2023-09-11 18:03:14,662 - modelscope - INFO - load ./output_training_data/
2023-09-11 18:03:14,920 - modelscope - INFO - Use user-specified model revision: v1.0.6
2023-09-11 18:03:16,164 - modelscope - INFO - am_config=./pretrain_work_dir/orig_model/basemodel_16k/sambert/config.yaml voc_config=./pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2023-09-11 18:03:16,165 - modelscope - INFO - audio_config=./pretrain_work_dir/orig_model/basemodel_16k/audio_config_se_16k.yaml
2023-09-11 18:03:16,165 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, './pretrain_work_dir/orig_model/basemodel_16k/sambert/ckpt/checkpoint_2400000.pth')])
2023-09-11 18:03:16,165 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, './pretrai

Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed


100%|██████████| 6/6 [00:00<00:00, 7605.27it/s]
INFO:root:TextScriptConvertor.process:
Save script to: ./pretrain_work_dir/data/Script.xml
INFO:root:TextScriptConvertor.process:
Save metafile to: ./pretrain_work_dir/data/raw_metafile.txt
INFO:root:[AudioProcessor] Initialize AudioProcessor.
INFO:root:[AudioProcessor] config params:
INFO:root:[AudioProcessor] wav_normalize: True
INFO:root:[AudioProcessor] trim_silence: True
INFO:root:[AudioProcessor] trim_silence_threshold_db: 60
INFO:root:[AudioProcessor] preemphasize: False
INFO:root:[AudioProcessor] sampling_rate: 16000
INFO:root:[AudioProcessor] hop_length: 200
INFO:root:[AudioProcessor] win_length: 1000
INFO:root:[AudioProcessor] n_fft: 2048
INFO:root:[AudioProcessor] n_mels: 80
INFO:root:[AudioProcessor] fmin: 0.0
INFO:root:[AudioProcessor] fmax: 8000.0
INFO:root:[AudioProcessor] phone_level_feature: True
INFO:root:[AudioProcessor] se_feature: True
INFO:root:[AudioProcessor] norm_type: mean_std
INFO:root:[AudioProcessor] max_norm:

推理 Inference （可编辑中文文本内容）

In [13]:
import os
from modelscope.models.audio.tts import SambertHifigan
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

model_dir = os.path.abspath("./pretrain_work_dir")

custom_infer_abs = {
    'voice_name':
    'F7',
    'am_ckpt':
    os.path.join(model_dir, 'tmp_am', 'ckpt'),
    'am_config':
    os.path.join(model_dir, 'tmp_am', 'config.yaml'),
    'voc_ckpt':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan', 'ckpt'),
    'voc_config':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan',
             'config.yaml'),
    'audio_config':
    os.path.join(model_dir, 'data', 'audio_config.yaml'),
    'se_file':
    os.path.join(model_dir, 'data', 'se', 'se.npy')
}
kwargs = {'custom_ckpt': custom_infer_abs}

model_id = SambertHifigan(os.path.join(model_dir, "orig_model"), **kwargs)

inference = pipeline(task=Tasks.text_to_speech, model=model_id)
output = inference(input="大家好呀，欢迎使用AI智能的声音克隆产品！") # 在这里编辑想要输出的文本

import IPython.display as ipd
ipd.Audio(output["output_wav"], rate=44100)

2023-09-11 18:05:28,285 - modelscope - INFO - am_config=/mnt/workspace/pretrain_work_dir/tmp_am/config.yaml voc_config=/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2023-09-11 18:05:28,286 - modelscope - INFO - audio_config=/mnt/workspace/pretrain_work_dir/data/audio_config.yaml
2023-09-11 18:05:28,286 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth'), (2400200, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400200.pth')])
2023-09-11 18:05:28,286 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/ckpt/checkpoint_2400000.pth')])
2023-09-11 18:05:28,287 - modelscope - INFO - se_path=/mnt/workspace/pretrain_work_dir/data/se/se.npy se_model_path=/mnt/workspace/pretrain_work_dir/orig_model/se.onnx
2023-09-11 18:05:28,287 - modelscope - INFO - mvn_path=/mnt/workspace/pretrain_work_dir/orig_model/mvn.npy
fe

Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Removing weight norm...


In [14]:
!pip install gradio

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 53.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.6/520.6 kB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 24.6 MB/s eta 0:00:00

In [16]:
def auto_label(audio):
  try:
    split_long_audio(whisper_model, audio, "test_wavs")
    os.makedirs("output_training_data", exist_ok=True)
    input_wav = "./test_wavs/"
    output_data = "./output_training_data/"
    ret, report = run_auto_label(input_wav=input_wav, work_dir=output_data, resource_revision="v1.0.7")

  except Exception:
    pass
  return "标注成功"

In [18]:
def train(a):
  try:
    os.makedirs("pretrain_work_dir", exist_ok=True)

    train_info = {
        TtsTrainType.TRAIN_TYPE_SAMBERT: {  # 配置训练AM（sambert）模型
            'train_steps': 52,               # 训练多少个step
            'save_interval_steps': 50,       # 每训练多少个step保存一次checkpoint
            'log_interval': 10               # 每训练多少个step打印一次训练日志
        }
    }

    # 配置训练参数，指定数据集，临时工作目录和train_info
    kwargs = dict(
        model=pretrained_model_id,                  # 指定要finetune的模型
        model_revision = "v1.0.6",
        work_dir=pretrain_work_dir,                 # 指定临时工作目录
        train_dataset=dataset_id,                   # 指定数据集id
        train_type=train_info                       # 指定要训练类型及参数
    )

    trainer = build_trainer(Trainers.speech_kantts_trainer,
                            default_args=kwargs)

    trainer.train()

  except Exception:
    pass

  return "训练完成"

In [20]:
import random

def infer(text):

  model_dir = os.path.abspath("./pretrain_work_dir")

  custom_infer_abs = {
      'voice_name':
      'F7',
      'am_ckpt':
      os.path.join(model_dir, 'tmp_am', 'ckpt'),
      'am_config':
      os.path.join(model_dir, 'tmp_am', 'config.yaml'),
      'voc_ckpt':
      os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan', 'ckpt'),
      'voc_config':
      os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan',
              'config.yaml'),
      'audio_config':
      os.path.join(model_dir, 'data', 'audio_config.yaml'),
      'se_file':
      os.path.join(model_dir, 'data', 'se', 'se.npy')
  }
  kwargs = {'custom_ckpt': custom_infer_abs}

  model_id = SambertHifigan(os.path.join(model_dir, "orig_model"), **kwargs)

  inference = pipeline(task=Tasks.text_to_speech, model=model_id)
  output = inference(input=text)

  filename = str(random.randint(1, 1000000000000))

  with open(filename + "myfile.wav", mode='bx') as f:
      f.write(output["output_wav"])
  return filename + "myfile.wav"

In [21]:
import gradio as gr
app = gr.Blocks()

with app:
    gr.Markdown("# 🥳🎶🎡 - 中文声音克隆")
    gr.Markdown("## 🌟 - 训练3分钟，推理5秒钟，中英自然发音 ")

    with gr.Row():
      inp1 = gr.Audio(type="filepath", label="请上传一段音频")
      out1 = gr.Textbox(label="标注情况", lines=1, interactive=False)

      out2 = gr.Textbox(label="训练情况", lines=1, interactive=False)
      inp2 = gr.Textbox(label="文本", lines=3)
      out3 = gr.Audio(type="filepath", label="合成的音频")
      btn1 = gr.Button("标注数据")
      btn2 = gr.Button("训练")
      btn3 = gr.Button("推理")

      btn1.click(auto_label, inp1, out1)
      btn2.click(train, out1, out2)
      btn3.click(infer, inp2, out3)

    gr.Markdown("### 注意❗：请不要生成会对个人以及组织造成侵害的内容，此程序仅供科研、学习及个人娱乐使用。")
app.launch(show_error=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


2023-09-11 18:16:15,173 - modelscope - INFO - am_config=/mnt/workspace/pretrain_work_dir/tmp_am/config.yaml voc_config=/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2023-09-11 18:16:15,174 - modelscope - INFO - audio_config=/mnt/workspace/pretrain_work_dir/data/audio_config.yaml
2023-09-11 18:16:15,174 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth'), (2400200, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400200.pth')])
2023-09-11 18:16:15,174 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/ckpt/checkpoint_2400000.pth')])
2023-09-11 18:16:15,175 - modelscope - INFO - se_path=/mnt/workspace/pretrain_work_dir/data/se/se.npy se_model_path=/mnt/workspace/pretrain_work_dir/orig_model/se.onnx
2023-09-11 18:16:15,175 - modelscope - INFO - mvn_path=/mnt/workspace/pretrain_work_dir/orig_model/mvn.npy
fe

Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Removing weight norm...


打包训练好的模型，可下载到本地

In [ ]:
!zip -r /model.zip ./pretrain_work_dir